<a href="https://colab.research.google.com/github/DavidMercadoFaustino/Analyzing-Data-with-Python---IBM-DA0101EN/blob/master/Modulo_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lidando com a missing values (NaN ou ?)
Solução:

1) **Excluir dado** - podemos excluir apenas o dado faltante ou toda a linha de entrado. Necessário atentar para aquela que vai ter menor impacto na analise

2)**Substituição** - Substituir o dado faltante pela média, pela frequencia ou pela analise dos outros dados.

3)**Deixar como faltante** - Muitas vezes é a melhor.

In [4]:
import pandas as pd
import numpy as np


In [5]:
df = pd.read_csv('/content/sample_data/automobile.csv')

In [6]:
df.describe(include="all")

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,bodystyle,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
count,205.000000,205.000000,164.000000,205,205,205,203,205,205,205,205.000000,205.000000,205.000000,205.000000,205.000000,205,205,205.000000,205,201.000000,201.000000,205.000000,203.000000,203.000000,205.000000,205.000000,201.000000
unique,NaN,NaN,NaN,22,2,2,2,5,3,2,NaN,NaN,NaN,NaN,NaN,7,7,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,toyota,gas,std,four,sedan,fwd,front,NaN,NaN,NaN,NaN,NaN,ohc,four,NaN,mpfi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,32,185,168,114,96,120,202,NaN,NaN,NaN,NaN,NaN,148,159,NaN,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,102.000000,0.834146,122.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.756585,174.049268,65.907805,53.724878,2555.565854,NaN,NaN,126.907317,NaN,3.329751,3.255423,10.142537,104.256158,5125.369458,25.219512,30.751220,13207.129353
std,59.322565,1.245307,35.442168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.021776,12.337289,2.145204,2.443522,520.680204,NaN,NaN,41.642693,NaN,0.273539,0.316717,3.972040,39.714369,479.334560,6.542142,6.886443,7947.066342
min,0.000000,-2.000000,65.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.600000,141.100000,60.300000,47.800000,1488.000000,NaN,NaN,61.000000,NaN,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,51.000000,0.000000,94.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.500000,166.300000,64.100000,52.000000,2145.000000,NaN,NaN,97.000000,NaN,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7775.000000
50%,102.000000,1.000000,115.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.000000,173.200000,65.500000,54.100000,2414.000000,NaN,NaN,120.000000,NaN,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,153.000000,2.000000,150.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.400000,183.100000,66.900000,55.500000,2935.000000,NaN,NaN,141.000000,NaN,3.590000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16500.000000


In [7]:
# Excluindo as linhas com celulas vazias usando .dropna()
df.dropna(subset=['price'],axis=0, inplace=True)


In [8]:
df['price']

0      13495.0
1      16500.0
2      16500.0
3      13950.0
4      17450.0
        ...   
200    16845.0
201    19045.0
202    21485.0
203    22470.0
204    22625.0
Name: price, Length: 201, dtype: float64

In [9]:
#Substituindo a celula vazia com média da coluna
mean = df['normalized-losses'].mean()

In [10]:
df['normalized-losses'].replace(np.nan, mean)

0      122.0
1      122.0
2      122.0
3      164.0
4      164.0
       ...  
200     95.0
201     95.0
202     95.0
203     95.0
204     95.0
Name: normalized-losses, Length: 201, dtype: float64

##Formatando DADOS
É necessario manter os dados padronizado para analizar os dados uma vez que assim ficam mais inteligiveis e facil de serem comparaveis e agregados.

In [11]:
df['city-mpg'] = 235/df['city-mpg']

In [12]:
df.rename(columns={'city-mpg':'city/100km'}, inplace=True)

In [13]:
df.dtypes

Unnamed: 0             int64
symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
bodystyle             object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city/100km           float64
highway-mpg            int64
price                float64
dtype: object

In [14]:
#Corrgindo o tipo dos dados
df['price'] = pd.to_numeric(df['price'])

## Colocando dados em mesma escala
Colocar na mesma escala é importante pois podemos comparar diferentes dimensões para que elas tenham o mesmo impacto nas analises estatisticas dos dados.

**Simples Escala** - Apenas dividir o valor pelo maior valor da dimensão. (Varia entre 0-1)

**Min-Max** - É a diferença do valor e do menor valor dividido pela diferença entre o maior valor e menor valor da dimensão (varia entre 0-1)

**z-score** - Diferença do valor pela média da dimensão dividido pelo desvio padrão (Varia entre -X a X, dependendo dos valores das variaveis)


In [15]:
#Aplicando a Escala Simples
#df['length'] = df['length']/df['length'].max()

In [16]:
#Aplicando o min-max
#df['length'] = (df['length'] - df['length'].min())/(df['length'].max() -df['length'].min()) 

In [17]:
#Aplicando o Z-score
#df['length'] = (df['length']-df['length'].mean())/(df['length'].std())

## Categorizando valores
Categorizar é bom para analise de dados com valores continuos, por exemplo peso altura. Mas também pode ser observado dados como idade para conseguir pegar faixas etárias e analizar melhor.

In [18]:
bins = np.linspace(min(df['price']), max(df['price']), 4)
group_names = ['Low', 'Medium','High']
df['price-binned'] = pd.cut(df['price'], bins, labels=group_names,include_lowest=True)

In [19]:
df['price-binned']

0         Low
1         Low
2         Low
3         Low
4         Low
        ...  
200       Low
201    Medium
202    Medium
203    Medium
204    Medium
Name: price-binned, Length: 201, dtype: category
Categories (3, object): [Low < Medium < High]

## Convertendo Variaveis Categoricas em Numericas
Variaveis categoricas em muitos casos tempos utiliza muito da memoria de processamento. Transformando em variaveis dummies (0,1) temos um precessamento mais eficiente.

In [20]:
pd.get_dummies(df['fuel-type'])

,diesel,gas
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
200,0,1
201,0,1
202,0,1
203,1,0


In [21]:
df.head(5)

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,bodystyle,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city/100km,highway-mpg,price,price-binned
0,0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,27,13495.0,Low
1,1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,27,16500.0,Low
2,2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,12.368421,26,16500.0,Low
3,3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,9.791667,30,13950.0,Low
4,4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,13.055556,22,17450.0,Low


## Agora Trabalhando no dataset

Primeiramente vamos buscar os valores que estão em faltante. Como durante a parte de cima consejá retiramos as linhas que não tinha o preço dos carros. Aqui iremos trabalhar com a substituição dos dados.

In [22]:
missing_data = df.isnull()
missing_data.head(5)

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,bodystyle,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city/100km,highway-mpg,price,price-binned
0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [24]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Unnamed: 0
False    201
Name: Unnamed: 0, dtype: int64

symboling
False    201
Name: symboling, dtype: int64

normalized-losses
False    164
True      37
Name: normalized-losses, dtype: int64

make
False    201
Name: make, dtype: int64

fuel-type
False    201
Name: fuel-type, dtype: int64

aspiration
False    201
Name: aspiration, dtype: int64

num-of-doors
False    199
True       2
Name: num-of-doors, dtype: int64

bodystyle
False    201
Name: bodystyle, dtype: int64

drive-wheels
False    201
Name: drive-wheels, dtype: int64

engine-location
False    201
Name: engine-location, dtype: int64

wheel-base
False    201
Name: wheel-base, dtype: int64

length
False    201
Name: length, dtype: int64

width
False    201
Name: width, dtype: int64

height
False    201
Name: height, dtype: int64

curb-weight
False    201
Name: curb-weight, dtype: int64

engine-type
False    201
Name: engine-type, dtype: int64

num-of-cylinders
False    201
Name: num-of-cylinders, dtype: int64

engine-size
False 

Aqui achamos as seguintes categorias com celulas vazias:

"normalized-losses": 37 missing data

"num-of-doors": 2 missing data

"bore": 4 missing data

"stroke" : 4 missing data

"horsepower": 2 missing data

"peak-rpm": 2 missing data


Para **normalized-losses**, **bore**, **stroke**, **horsepower**, **peak-rpm** será utilizado a substituição pela média dos valores da coluna.

Para **num-of-doors** iremos utilizar a frequência.

In [33]:
media_norm_loss = df["normalized-losses"].mean(axis=0)
print('Média Normal Loss:', media_norm_loss)
media_bore = df['bore'].mean(axis=0)
print('Média Bore:', media_bore)
media_stroke = df['stroke'].mean(axis=0)
print('Média Stroke:', media_stroke)
media_horsepower = df['horsepower'].mean(axis=0)
print('Média Horsepower:', media_horsepower)

Média Normal Loss: 122.0
Média Bore: 3.3307106598984775
Média Stroke: 3.2569035532994857
Média Horsepower: 103.39698492462311


In [34]:
#Substituindo as celulas vazias pela média dos valores
df["normalized-losses"].replace(np.nan, media_norm_loss, inplace=True)
df["bore"].replace(np.nan, media_bore, inplace=True)
df["stroke"].replace(np.nan, media_stroke, inplace=True)
df["horsepower"].replace(np.nan, media_horsepower, inplace=True)

In [37]:
df.describe()

,Unnamed: 0,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city/100km,highway-mpg,price
count,201.000000,201.000000,201.00000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,199.000000,201.000000,201.000000,201.000000
mean,102.900498,0.840796,122.00000,98.797015,174.200995,65.889055,53.766667,2555.666667,126.875622,3.330711,3.256904,10.164279,103.396985,5117.587940,9.944145,30.686567,13207.129353
std,59.234956,1.254802,31.99625,6.066366,12.322175,2.101471,2.447822,517.296727,41.546834,0.268072,0.316048,4.004965,37.365602,480.521824,2.534599,6.815150,7947.066342
min,0.000000,-2.000000,65.00000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,4.795918,16.000000,5118.000000
25%,53.000000,0.000000,101.00000,94.500000,166.800000,64.100000,52.000000,2169.000000,98.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,7.833333,25.000000,7775.000000
50%,103.000000,1.000000,122.00000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,9.791667,30.000000,10295.000000
75%,154.000000,2.000000,137.00000,102.400000,183.500000,66.600000,55.500000,2926.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,12.368421,34.000000,16500.000000
max,204.000000,3.000000,256.00000,120.900000,208.100000,72.000000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,262.000000,6600.000000,18.076923,54.000000,45400.000000


Substituindo pela frequência

In [40]:
#podemos buscar pela função value_counts
df['num-of-doors'].value_counts()


four    113
two      86
Name: num-of-doors, dtype: int64

In [41]:
#Ou podemos usar pela .idxmax()
df['num-of-doors'].value_counts().idxmax()


'four'

In [42]:
df["num-of-doors"].replace(np.nan, "four", inplace=True)

Agora padronizando de acordo com o sistema métrico

In [43]:
#Transformando Miles por Galão para Litros Por Km
df["highway-mpg"] = 235/df["highway-mpg"]

In [46]:
df.rename(columns={'"highway-mpg"':'highway-L/100km'}, inplace=True)

In [47]:
df.head(5)

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,bodystyle,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city/100km,highway-mpg,price,price-binned
0,0,3,122.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,8.703704,13495.0,Low
1,1,3,122.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,8.703704,16500.0,Low
2,2,1,122.0,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,12.368421,9.038462,16500.0,Low
3,3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,9.791667,7.833333,13950.0,Low
4,4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,13.055556,10.681818,17450.0,Low


Colocando as variáveis na mesma escala

Agora vamos colocar as variaveis **height**, **width**, **length** na mesma escala para melhor observar o comportamento dela.



In [50]:
#Utilizaremos o método mais simples
df['length'] = df['length']/df['length'].max()
df['width'] = df['width']/df['width'].max()
df['height'] = df['height']/df['height'].max()

In [53]:
df.head(5)

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,bodystyle,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city/100km,highway-mpg,price,price-binned
0,0,3,122.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,8.703704,13495.0,Low
1,1,3,122.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,8.703704,16500.0,Low
2,2,1,122.0,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,0.876254,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,12.368421,9.038462,16500.0,Low
3,3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,0.848630,0.919444,0.908027,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,9.791667,7.833333,13950.0,Low
4,4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,0.848630,0.922222,0.908027,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,13.055556,10.681818,17450.0,Low


Categorizando variaveis continuas. 

Vamos criar categorias com valores agrupamos.

In [55]:
#Criando as categorias de acordo com as faixas de valores
bins2 = np.linspace(min(df["horsepower"]), max(df["horsepower"]), 4)
bins2

array([ 48.        , 119.33333333, 190.66666667, 262.        ])

In [56]:
categoria_nomes = ['Low','Medium','High']

In [59]:
df['horsepower-binned'] = pd.cut(df['horsepower'], bins2, labels=group_names, include_lowest=True )
df[['horsepower','horsepower-binned']].head(5)

,horsepower,horsepower-binned
0,111.0,Low
1,111.0,Low
2,154.0,Medium
3,102.0,Low
4,115.0,Low


In [60]:
df["horsepower-binned"].value_counts()

Low       153
Medium     43
High        5
Name: horsepower-binned, dtype: int64

Transformando em variaveis dummies

Usaremos para o **Fuel-type**, **Apriration**, **Body style**

In [66]:
#Transformando em variavel Dummy
dummy_variable_1 = pd.get_dummies(df["fuel-type"])
dummy_variable_1.head(5)

,diesel,gas
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [67]:
#Mudando o nome das colunas para melhor compreensão
dummy_variable_1.rename(columns={'fuel-type-diesel':'gas', 'fuel-type-diesel':'diesel'}, inplace=True)
dummy_variable_1.head(5)

,diesel,gas
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [68]:
#Juntando as tabelas
df = pd.concat([df, dummy_variable_1], axis=1)

In [ ]:
#Excluindo a coluna original
df.drop("fuel-type", axis = 1, inplace=True)

In [72]:
df.head(5)

,Unnamed: 0,symboling,normalized-losses,make,aspiration,num-of-doors,bodystyle,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city/100km,highway-mpg,price,price-binned,horsepower-binned,diesel,gas
0,0,3,122.0,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,8.703704,13495.0,Low,Low,0,1
1,1,3,122.0,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,11.190476,8.703704,16500.0,Low,Low,0,1
2,2,1,122.0,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,0.876254,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,12.368421,9.038462,16500.0,Low,Medium,0,1
3,3,2,164.0,audi,std,four,sedan,fwd,front,99.8,0.848630,0.919444,0.908027,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,9.791667,7.833333,13950.0,Low,Low,0,1
4,4,2,164.0,audi,std,four,sedan,4wd,front,99.4,0.848630,0.922222,0.908027,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,13.055556,10.681818,17450.0,Low,Low,0,1


In [74]:
# Agora usando a variavel Aspiration
dummy_variable_2 = pd.get_dummies(df['aspiration'])
dummy_variable_2.rename(columns={'std':'aspiration-std', 'turbo': 'aspiration-turbo'}, inplace=True)
dummy_variable_2.head()

,aspiration-std,aspiration-turbo
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [75]:
df = pd.concat([df, dummy_variable_2], axis=1)
df.drop('aspiration', axis = 1, inplace=True)

In [76]:
#Agora usando a variavel Body Style
dummy_variable_3 = pd.get_dummies(df['bodystyle'])
dummy_variable_3.head()

,convertible,hardtop,hatchback,sedan,wagon
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0


In [77]:
dummy_variable_3.rename(columns={'convertible':'convertible-style', 'hardtop': 'hardtop-style','hatchback':'hatchback-style','sedan':'sedan-style','wagon':'wagon-style'}, inplace=True)
dummy_variable_3.head()

,convertible-style,hardtop-style,hatchback-style,sedan-style,wagon-style
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0


In [78]:
df = pd.concat([df, dummy_variable_3], axis=1)
df.drop('bodystyle', axis = 1, inplace=True)

Agora salvar o arquivo já com toda a Limpeza realizada

In [80]:
df.to_csv('/content/sample_data/automovile_clean_df.csv')

## Agora estamos prontos para as futuras analises.